<a href="https://colab.research.google.com/github/Untick/resume-builder-gr2/blob/Labzin-Vyacheslav-branch/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_%D0%B8%D0%B7_%D0%B4%D0%B8%D0%B0%D0%BB%D0%BE%D0%B3%D0%B0_%D0%B8_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%BD%D0%B0_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%BE_%D0%B7%D0%B0%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import openai
import getpass
import os
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [24]:
import re
import requests

# Создание резюме из диалога

In [11]:
# Инструкция для GPT, которая будет подаваться в system
system_doc= '''Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
У тебя большой опыт по составлению грамотных резюме, чтобы они привлекали внимание работодателей в IT сфере.
У тебя есть образец резюме.

К тебе будет поступать информация клиента. Информация клиента будет представлена в виде словаря.
Из представленной информации составь грамотное резюме.

При составлении резюме добавь переносы строк, чтобы резюме выглядело также, как образец резюме.
 '''

In [3]:
with open('resume.txt', 'r') as file:
  dialog = file.read()

In [4]:
print (dialog)

 {"1 ФИО": "Иванов Вячеслав Владимирович"} {"2 краткое описание вашей профессиональной роли или цели": "Data Scientist"} 
 {"3 номер телефона": " "} {"4 адрес электронной почты": "Ivanov@mail.ru"} {"5 О себе": " "}
 {"6 ключевые технологии, с которыми работал клиент": "Python, R, SQL, машинное обучение"}
 {"7 опыт работы - проекты, в которых клиент принимал участие": "Создание нейро-консультанта"} {"8 достижения - сертификаты, награды, если они есть у клиента": "Занял
 третье место в хакатоне по искусственному интеллекту"} {"9 опыт работы - название компании": "ПАО ГАЗ"} {"10 опыт работы - должность": "Главный технолог"}
 {"11 период работы": "с июля 2015 года по настоящее время"} {'12 краткое описание обязанностей и достижений': " "}
 {"13 образование - название учебного заведения": "Южный федеральный университет"} {"14 специализация в учебном заведении": "Прикладная математика и информатика"} 
 {"15 год окончания учебного заведения": "2011"} {"16 дополнительные курсы или сертификаты,

In [28]:
with open('resume_sample.txt', 'r') as file:
  resume_sample = file.read()

In [29]:
print (resume_sample)

﻿Вячеслав Лабзин  
Data Scientist
КОНТАКТЫ  
  

+7(499)-123-47-85                  
labzin@mail.ru  


О СЕБЕ


Я Junior специалист в сфере искусственного интеллекта. Имею опыт работы с анализом данных, разработкой нейронных сетей. 
Активно изучаю и работаю с ChatGPT. 


НАВЫКИ


Python, GPT (языковое моделирование), NumPy, Pandas, Matplotlib, Seaborn, Keras,Tensorflow 


ПРОФЕССИОНАЛЬНАЯ ДЕЯТЕЛЬНОСТЬ


Проект: Создание веб-сайта для генерации резюме и сопроводительных писем



Проект: Разработка нейронной сети для обработки данных горного эндоскопа



2016 - сейчас
ООО «КЗ «Ростсельмаш»


ГЛАВНЫЙ СПЕЦИАЛИСТ МАТЕРИАЛЬНО-ТЕХНИЧЕСКОГО СНАБЖЕНИЯ ПО ЦЕНООБРАЗОВАНИЮ


* проверка и согласование цен на покупные товарно-материальные ценности




ОБРАЗОВАНИЕ


2023
УНИВЕРСИТЕТ ИСКУССТВЕННОГО ИНТЕЛЛЕКТА
* Chat GPT Professional
* Data Science и нейронные сети



In [9]:
# Для подачи в user-assistant, чтобы показать пример, какой диалог будет подан в user
dialog_sample = '''
{'1 ФИО': 'Вячеслав Лабзин'} {'2 краткое описание вашей профессиональной роли или цели, например "Full Stack разработчик" или "Data Scientist"': 'Data Scientist'}
{'3 номер телефона': '+7(499)-123-47-85'} {'4 адрес электронной почты': 'labzin@mail.ru'}
{'5 О себе': 'Я Junior специалист в сфере искусственного интеллекта. Имею опыт работы с анализом данных, разработкой нейронных сетей.
Активно изучаю и работаю с ChatGPT.'}
{'6 ключевые технологии, с которыми работал клиент. Надо выбрать из перечня: Python, R, SQL, базовые знания статистики, машинное обучение, обработка данных, программная разработка': 'Python, GPT (языковое моделирование), NumPy, Pandas, Matplotlib, Seaborn, Keras,Tensorflow'},
{'7 опыт работы - проекты, в которых клиент принимал участие': 'Создание веб-сайта для генерации резюме и сопроводительных писем; Разработка нейронной сети для обработки данных горного эндоскопа'},
{'8 достижения - сертификаты, награды, если они есть у клиента': ' '} {'9 опыт работы - название компании': 'ООО «КЗ «Ростсельмаш»'}
{'10 опыт работы - должность': 'ГЛАВНЫЙ СПЕЦИАЛИСТ МАТЕРИАЛЬНО-ТЕХНИЧЕСКОГО СНАБЖЕНИЯ ПО ЦЕНООБРАЗОВАНИЮ'} {'11 период работы': '2016 - сейчас'}
{'12 краткое описание обязанностей и достижений': 'проверка и согласование цен на покупные товарно-материальные ценности'}
{'13 образование - название учебного заведения': 'УНИВЕРСИТЕТ ИСКУССТВЕННОГО ИНТЕЛЛЕКТА'} {'14 специализация в учебном заведении': 'Chat GPT Professional; Data Science и нейронные сети'}
{'15 год окончания учебного заведения': '2023'}
'''

In [17]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f'{system_doc} \n\nОбразец резюме: {resume_sample}'},
        {"role": "user", "content": f"Составь резюме из диалога: \n\n{dialog_sample}. Информацию для резюме бери только из диалога. Если в диалоге информации нет, ставь пропуски. От себя ничего не добавляй."},
        {"role": "assistant", "content": resume_sample},
        {"role": "user", "content": f"Составь резюме из диалога: \n\n{dialog}. Информацию для резюме бери только из диалога. Если в диалоге информации нет, ставь пропуски. От себя ничего не добавляй."}
    ]
)
answer = response['choices'][0]['message']['content']
print(answer)

﻿Иванов Вячеслав Владимирович  
Data Scientist
КОНТАКТЫ  
  

+7(XXX)XXXXXXX                  
Ivanov@mail.ru  


О СЕБЕ


 


НАВЫКИ


Python, R, SQL, машинное обучение 


ПРОФЕССИОНАЛЬНАЯ ДЕЯТЕЛЬНОСТЬ


Проект: Создание нейро-консультанта



Третье место в хакатоне по искусственному интеллекту




2015 - настоящее время
ПАО ГАЗ


Главный технолог


* 


ОБРАЗОВАНИЕ


2011
Южный федеральный университет
* Прикладная математика и информатика


ДОПОЛНИТЕЛЬНЫЕ КУРСЫ И СЕРТИФИКАТЫ


курс по машинному обучению


# Проверка заполнения резюме

In [19]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [20]:
# Функция для представления длинного ответа в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    """
    Функция разбивает длинный текст на строки определенной максимальной длины.
    """
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [25]:
# ПРОМПТ в system
system = load_document_text('https://docs.google.com/document/d/12fSAEOmhbkLTHrDq84OCI8Myo0HnHQ575yh02lzCNI8/edit?usp=sharing')

In [30]:
# Образец заполненного резюме
resume_complet = load_document_text(
    'https://docs.google.com/document/d/1UeR8y8husKJCVT5M-mQ_N8rKk-BKVAex/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [27]:
# Резюме без образования
resume_none_educ = load_document_text(
    'https://docs.google.com/document/d/1qBuuFUhOPLqpIhfX5-ZdrhN8YVtzOrF3/edit?usp=sharing&ouid=113312547144364951406&rtpof=true&sd=true')

In [31]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"{system} \n\nОбразец правильно составленного резюме: {resume_complet}"},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {resume_none_educ}."},
        {"role": "assistant", "content": "В резюме не заполнен блок Образование. Добавьте свое образование, связанное с IT специальностями."},
        {"role": "user", "content": f"Кратко укажи замечания по заполнению резюме: {answer}."}
    ]
)
remarks = insert_newlines(response['choices'][0]['message']['content'])
print(remarks)

 Замечания по заполнению резюме: - В блоке "О себе" не заполнена информация. Добавьте информацию о себе, своих целях и квалификации в сфере Data Science. - В блоке
 "Навыки" нет указания на IT навыки. Укажите, какие конкретно навыки связаны с IT и Data Science. Примеры IT навыков: Python, R, SQL, машинное обучение. - В блоке
 "Профессиональная деятельность" нет описания проекта "Создание нейро-консультанта". Добавьте информацию о задачах и результате проекта. - В блоке "Профессиональная
 деятельность" нет описания третьего места в хакатоне по искусственному интеллекту. Добавьте информацию о хакатоне и вашем вкладе в достижение третьего места. - В блоке
 "Профессиональная деятельность" не указаны обязанности и достижения в качестве главного технолога в ПАО ГАЗ. Добавьте информацию о ваших ответственностях и достижениях в
 данной должности. - В блоке "Образование" не указан год окончания университета. Добавьте год окончания. - В блоке "Дополнительные курсы и сертификаты" не указано
 назв